# TODOs


In [ ]:
library(forecast)

In [ ]:
loadData <- function(dataFolder) {
    files <- list.files(dataFolder)
    data <- list()
    for(file in files) {    
        df <- read.csv(paste0(dataFolder, "/", file), stringsAsFactors=F)    
        minYear <- min(df$Year)
        complaintType <- substr(file,1,(nchar(file))-4)    
        tsObject <- ts(df$Complaints, start=c(minYear, 1), frequency = 12)
        data[[complaintType]] <- tsObject
    }
    data
}
data <- loadData("../../data/topNComplaints")
# change dates if data changes
train_start <- c(2015, 6)
test_start <- c(2015, 7)

In [ ]:
series <- data[["Dog menace "]]

In [ ]:
tsdisplay(series)

In [ ]:
# data before 2012 are too few to consider
series <- window(series, start=c(2012, 4), end=c(2016, 6))
tsdisplay(series)

## Cleaning up data 

This data looks like it has outliers near the end of 2016. Let's take a look at the series if these (possible) outliers were winsorized

In [ ]:
plot(series, col="red", lty=2)
lines(tsclean(series), lty=1)
legend("topright", col=c("red", "black"), lty=c(2,1), legend=c("Original", "Cleaned"))

There are no significant outliers, and the data is clean. 

In [ ]:
# split the data into train and testing sets for further analysis
series.train <- window(series, end=train_start) # this will be used in all subsequent analysis
series.test <- window(series, start=test_start)

## Decomposition
Let's see if there is a seasonal component in this data. 

In [ ]:
plot(stl(series.train, s.window="periodic"))
# The series does look like it has a seasonal component - let's take a look at that.

In [ ]:
plot(stl(series, s.window=6)) # change s.window to something that make sense

In [ ]:
# let's take a look at which month this series peaks
# looks like there are two spikes, one in march and the other in september
# the data does contain another spike in between the two - which isn't apparent in the 
# seasonal component
seasonal <- stl(series, s.window=6)$time.series[, 1] # change s.window
plot(seasonal, col="grey")
month1 <- 9 # september
month2 <- 3 # march
for(i in 2012:2016) {    
    abline(v=(month1-1)/12 + i, lty=2)
    abline(v=(month2-1)/12 + i, lty=3)
}

In [ ]:
# let's superimpose this on the original data, to see if this effect 
# is significant
plot(series, col="grey")
month1 <- 9
month2 <- 3
for(i in 2012:2016) {    
    abline(v=(month1-1)/12 + i, lty=2)
    abline(v=(month2-1)/12 + i, lty=3)
}

In [ ]:
# this series looks like it fits the data well - since the seasonal component does seem to increase as time progresses
# let's set s.window = "periodic"
stl.fit <- stl(series, s.window="periodic")
series.adj <- seasadj(stl.fit)
tsdisplay(series.adj)

## Forecasting
### ARIMA models - estimating p, d, q

First, let us estimate $d$. This is done by looking at the ACF of the data.

In [ ]:
Acf(series.adj)

In [ ]:
# the above series is a classic example of a series that requires a diff of order 1, 
# so let's try that out and take a look at the Acf to see if it is overdifferenced
tsdisplay(diff(series.adj, lag=1, differences = 1))

In [ ]:
# looks like the series has a strong, negative ACF at lag12 
# indicating a possible seasonality?# 
# let's also look at d=2
tsdisplay(diff(series.adj, lag = 1, differences = 2))
# this is clearly overdifferenced, so d <= 1

In [ ]:
# take a look at standard-deviation
sd.0 <- sd(series.adj)
sd.1 <- sd(diff(series.adj, differences = 1))
sd.2 <- sd(diff(series.adj, differences = 2))
print(paste0("SD with d = 0: ", sd.0, ", SD with d = 1: ", sd.1, ", SD with d = 2: ", sd.2))
# in terms of sd, d=1 is a better fit

In [ ]:
series.diff <- diff(series.adj, lag=1, differences = 1)

In [ ]:
plot(series.diff, col="grey")
# a 2x4 MA
lines(ma(ma(series.diff, order=2), order=4))
abline(mean(series.diff), 0, col="blue", lty=2)

In [ ]:
ndiffs(series.adj)

Next, we need to estimate p and q. To do this, we take a look at the PACF of the data. Note that this analysis is done on the differenced data. If we decide to fit a model with d=0, then we need to perform this analysis for the un-differenced data as well

In [ ]:
# let d=0 first
# looks like a AR(1), MA(0)
Pacf(series.adj)

In [ ]:
# let's try with d=1
# looks like MA(12) process, possibly an AR(3)
Pacf(series.diff)

### Building candidate models

In [ ]:
modelArima <- function(series, order, h, testData = NULL) {
    fit <- Arima(series, order=order)
    print(summary(fit))
    predictions <- forecast(fit, h)
    # compute max and min y
    min.yvalue <- min(min(series), min(testData))
    max.yvalue <- max(max(series), max(testData))
    
    plot(predictions, ylim=c(min.yvalue, max.yvalue))
    if(!is.null(testData)) {
        lines(testData, col="red", lty=2)
        print(accuracy(predictions, testData))
    }
    # check if residuals looklike white noise
    Acf(residuals(fit), main="Residuals")
    # portmantaeu test
    print(Box.test(residuals(fit), lag=24, fitdf=4, type="Ljung"))
}

In [ ]:
# split the series into a test and a train set
series.train <- window(series.adj, end=c(2015, 6))
series.test <- window(series.adj, start=c(2015, 7))

In [ ]:
modelArima(series.train, c(1, 0, 13), length(series.test), series.test)

In [ ]:
# with d=1, p=3, q=12
modelArima(series.train, c(3, 1, 12), length(series.test), series.test)

In [ ]:
# since the results are somewhat inaccurate
# let's look at the original data itself, and estimate (p, d, q)
# let's first estimate d
Acf(series)

In [ ]:
# d = 1
# let's try differencing
Acf(diff(series, differences = 1))
# looks good, let's try d=2
Acf(diff(series, differences = 2))
# since there's a very strong negative correlation at lag1, we can 
# conclude that this series is overdifferenced, so we can select d <=1

In [ ]:
Pacf(series) # looks like a MA(1)
Pacf(diff(series, differences = 1)) # looks like a MA(14)